In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

# hard coded nrows
df_all = pd.read_csv('../../../diploma/multi-source-syber-security-events/flows.txt', header=None, nrows=20000)

df_all.columns = ['time', 'duration', 'source computer', 'source port', 'destination computer', 
              'destination port', 'protocol', 'packet count', 'byte count']

df = df_all

df.index = df['time']
df.drop(columns=['time'],inplace=True)
df.head()

,duration,source computer,source port,destination computer,destination port,protocol,packet count,byte count
time,,,,,,,,
1,0,C1065,389,C3799,N10451,6,10,5323
1,0,C1423,N1136,C1707,N1,6,5,847
1,0,C1423,N1142,C1707,N1,6,5,847
1,0,C14909,N8191,C5720,2049,6,1,52
1,0,C14909,N8192,C5720,2049,6,1,52


In [ ]:
from helper import group_scale_data, group_scale_data_batch

groupped_data, hosts, parameters = group_scale_data(df, size_of_bin_seconds=60,
                                                    addZeros=True, doScale=True, scaler='log', verbose=1)

A total of 13 time epochs have been encountered
Done with 0 hosts out of 1576
Done with 78 hosts out of 1576
Done with 156 hosts out of 1576
Done with 234 hosts out of 1576
Done with 312 hosts out of 1576
Done with 390 hosts out of 1576


In [ ]:
groupped_data.head()

In [ ]:
from emClustering import OnlineEM
from plots import plot_points, plot_results, plot_category, plot_all_categories
from kplusplus import KPlusPlus

In [ ]:
plot_points(groupped_data.values[:, :2])

In [ ]:
mixtures = 15

kplusplus = KPlusPlus(mixtures, groupped_data.values[:, :2], stochastic=False, stochastic_n_samples=3000)

kplusplus.init_centers(verbose=1)

# random initialization
onlineEM = OnlineEM([1/mixtures]*mixtures, kplusplus.centers, 500, n_clusters=8, verbose=1, update_power=0.5)

plot_points(groupped_data.values[:, :2], onlineEM)

In [ ]:
data = groupped_data.values[:,[0,1,3]]

np.random.shuffle(data)


onlineEM.fit(data)

In [ ]:
plot_points(groupped_data.values[:, :2], onlineEM)